# Métricas de evaluación para regresión

La forma más común de evaluar el ajuste global de un modelo lineal es por el valor $R^2$. $R^2$ es la proporción de la varianza explicada, es decir, la proporción de la varianza en los datos observados que es explicada por el modelo, o la reducción del error sobre el modelo nulo. El modelo nulo sólo predice la media de la respuesta observada, y por lo tanto tiene un *intercept* pero no una pendiente.

$R^2$ está entre 0 y 1, y más alto es mejor porque significa que el modelo explica más varianza.

Además de $R^2$ existen otras métricas para evaluar la calidad de los modelos de regresión. Las más utilizadas son las siguientes:

* **Mean Absolute Error (MAE)** es la media de los valores absolutos de los errores
$$\frac{1}{n}\sum^{n}_{i=1}\left | y_i - \widehat{y}_i  \right |$$

* **Mean Squared Error (MSE)** es la media de los cuadrados de los errores
$$\frac{1}{n}\sum^{n}_{i=1}\left ( y_i - \widehat{y}_i  \right )^2$$

* **Root Mean Squared Error (RMSE)** es la raíz cuadrada de la media de los errores al cuadrado
$$\sqrt{\frac{1}{n}\sum^{n}_{i=1}\left ( y_i - \widehat{y}_i  \right )^2}$$

Todas estas métricas están implementadas en [scikit-learn](https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics).

El *MSE* es más popular que el *MAE* porque "castiga" los errores más grandes. Esto quiere decir que es una métrica más exigente con respecto a la calidad del modelo. Por otro lado, la *RMSE* es aún más popular que la *MSE* porque sus resultados son interpretables en unidades de la variable de salida.

Observemos su funcionamiento sobre el conjunto de datos de las casas de Boston:

In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error

X, y = load_boston(return_X_y=True)

ridge = Ridge(alpha=1.0).fit(X, y)

y_pred = ridge.predict(X)

print("MAE: ", mean_absolute_error(y, y_pred))
print("MSE: ", mean_squared_error(y, y_pred))
print("RMSE: ", mean_squared_error(y, y_pred, squared=False))
print("R2: ", ridge.score(X, y))

MAE:  3.2645739551644364
MSE:  22.044452243088205
RMSE:  4.695151993608748
R2:  0.7388703133867615


Observamos que el MAE es el error más interpretable, puesto de promedio nos equivocamos en 3200 dólares en el precio de una casa. El MSE no tiene un significado fácil de interpretar, pero el RMSE muestra valores similares al MAE, solo que, al ser más alto, denota que el error en las predicciones es dispar. Por último, el $R^2$ demuestra que se está explicando el 73\% de la varianza.

## Evaluación justa de los modelos

Hasta ahora hemos estado calculando las medidas de calidad de los modelos con el mismo conjunto de datos que se ha usado para su entrenamiento. Si bien evaluar los modelos de esta forma nos puede dar una aproximación de la dificultad que ha tenido el modelo para entrenarse, es una evaluación **nada justa** del modelo, puesto que estamos evaluando su rendimiento pidiendo que nos calcule la predicción de **los mismos datos** con los que ha realizado el ajuste. Por tanto, estamos confundiendo **rendimiento** con **overfitting**.

Para realizar una evaluación más justa de cualquier modelo de *machine learning* se utiliza la división del conjunto de datos en dos conjuntos: uno para **entrenamiento** y otro para **test**.

La idea básica es reservar un porcentaje de los datos de entrada para utilizarlos posteriormente en la fase de evaluación del modelo. Estos datos, al no participar en el entrenamiento de este, pueden considerarse como datos futuros. Por lo tanto, comprobar qué tal se comporta el modelo intentando predecir los datos de **test** es una forma justa de evaluar la capacidad de generalizar del modelo, lo que a fin de cuentas es la principal característica cuando se trata de *machine learning*.

Un detalle importante es el mecanismo utilizado para realizar la división del conjunto de datos. Si se realiza *a mano* estamos inevitablemente introduciendo un sesgo en el proceso, ya que podría suceder que el subconjunto seleccionado no sea representativo del conjunto total. Por ello se suele dividir el conjunto de datos mediante un muestreo **aleatorio**.

`sklearn` cuenta con una función para realizar la partición mediante muestreo aleatorio del conjunto de datos de entrada en dos subconjuntos, uno de entrenamiento y otro de test. La función en cuestión es `train_test_split`, y se encuentra en el módulo `model_selection`.

Vamos a repetir la evaluación de las combinaciones de variables para el conjunto de datos del precio de las casas, aunque utilizando la división de los datos en entrenamiento y test:

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error

X, y = load_boston(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

ridge = Ridge(alpha=1.0).fit(X_train, y_train)

y_pred = ridge.predict(X_test)

print("MAE: ", mean_absolute_error(y_test, y_pred))
print("MSE: ", mean_squared_error(y_test, y_pred))
print("RMSE: ", mean_squared_error(y_test, y_pred, squared=False))
print("R2: ", ridge.score(X_test, y_test))

MAE:  3.17853907603455
MSE:  22.044053089860952
RMSE:  4.6951094864615195
R2:  0.7041586727559442


## Cross-validation (validación cruzada)

Aunque hemos dado un gran paso hacia una evaluación más justa de los modelos, todavía hay algunos detalles que se pueden mejorar. Y es que, si la separación del *dataset* en entrenamiento y test se realiza mediante un muestreo aleatorio, podría suceder que por casualidad todas las muestras que vayan al conjunto de test sean todas casas de 3 habitaciones, por poner un ejemplo. De esta forma se estaría introduciendo un sesgo en el proceso.

Una forma de mitigar esto es realizar $K$ separaciones diferentes y aleatorias, calcular las medidas de calidad de cada una de estas separaciones y luego agruparlas con alguna función de agregación (media, mediana, ...). Este proceso se conoce como **K-fold cross-validation** y se puede considerar casi un estándar a la hora de evaluar el rendimiento de un modelo de aprendizaje computacional.

La siguiente imagen muestra un proceso de 5-fold cross-validation, donde se observa que cada iteración usa unos conjuntos de entrenamiento y test distintos:

![](https://i.imgur.com/R6nj4E9.png)

`sklearn` cuenta con herramientas para realizar validación cruzada a la hora de evaluar los modelos. A continuación vamos a realizar la evaluación del modelo que tiene en cuenta todas las variables de entrada siguiendo el enfoque de **validación cruzada**:

In [3]:
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

X, y = load_boston(return_X_y=True)
ridge = Ridge(alpha=1)

In [4]:
cross_val_score(ridge, X, y, cv=5, scoring='neg_mean_squared_error')

array([-11.7110334 , -23.58316804, -29.69339407, -80.22988232,
       -31.11617767])

Hay otras métricas interesantes, como por ejemplo el error máximo cometido en alguna de las predicciones, que nos da una medida de cómo de 'malo' es nuestro modelo en el peor de los casos:

In [5]:
cross_val_score(ridge, X, y, cv=5, scoring='max_error')

array([-10.87844828, -15.35836041, -15.79933142, -36.99334136,
       -14.8295804 ])

O el error absoluto mediano, que es una métrica más robusta con respecto a las observaciones atípicas, también conocidas como *outliers*:

In [6]:
cross_val_score(ridge, X, y, cv=5, scoring='neg_median_absolute_error')

array([-1.72771703, -2.80656206, -3.30624995, -2.64389689, -4.93110367])

Para conjunto de datos muy reducidos, podemos usar *leave one out*, realiza tanto entrenamientos como muestras tenga el conjunto de datos excluyendo, en cada uno de ellos, una muestra que luego será evaluada:

In [9]:
from sklearn.model_selection import LeaveOneOut
import numpy as np

scores = cross_val_score(ridge, X, y, cv=X.shape[0], scoring='neg_median_absolute_error')
np.mean(scores)

-3.3739675201395656

---

Creado por **Raúl Lara** (raul.lara@upm.es) y **Fernando Ortega** (fernando.ortega@upm.es)

<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png">